*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher<br>
**Date:**  06.02.2024

--- 

# Calculate hot vehicle emissions using HBEFA emission factors

<!--Notebook description and usage information-->
This notebook implements the <utls/hot_emission_process.py> function and multiprocessing to calculate hot vehicle emissions for a given area. 


In [35]:
import sys
import multiprocessing
import geopandas as gpd
import pandas as pd
from datetime import datetime

sys.path.append('../utils')
import data_paths
from traffic_counts import TrafficCounts
from hbefa_hot_emissions import HbefaHotEmissions
from hot_emission_process import process_daily_emissions, process_hourly_emissions

# Reload local modules on changes
%reload_ext autoreload
%autoreload 2

# Notebook Settings

In [100]:
# Define start and end time for emission calculation. Ideally this should cover a whole year.
start_date = datetime(2019, 1, 1)
end_date = datetime(2022, 12, 31)

# define filename of the visum file
visum_filename = "visum_links.GPKG"

# if True, the script will only calculate the emission for the area within the roi polygon
clip_to_area = False
roi_polygon = data_paths.MUNICH_BOARDERS_FILE # defines ROI for clipping

# select aggregated or los-specific mode for traffic situation calculation
#mode = 'aggregated' 
mode = 'los_specific'

# if True, the timeprofiles for the selected components will be calculated
calculate_timeprofile = True
store_timeprofiles = True

# define number of processes for multiprocessing
NUMBER_OF_PROCESSES = 7

###
#
# STORE RESULTS
#
###

store_results = True
store_filename = f'linesource_Munich_2019_new_vc.gpkg'

## Import Data and Initialize Objects

In [37]:
# import visum model
visum = gpd.read_file(data_paths.VISUM_FOLDER_PATH + visum_filename)

if clip_to_area:
    roi = gpd.read_file(roi_polygon).to_crs(visum.crs)
    visum = gpd.clip(visum, roi)
    visum = visum.explode(ignore_index=True) # convert multipolygons to polygons

#visum = visum_links
visum = visum.reset_index(drop = True).reset_index() # reset index for calculation

# initialize traffic cycles
cycles = TrafficCounts()
# initialize HBEFA emission factors
hbefa = HbefaHotEmissions()

Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_PC.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_LCV.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_HGV.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_Coach.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_MOT.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/ef_aggregated_los/EFA_HOT_Vehcat

## Process Inventory
Use multiprocessing to calculate the emission for each road link day by day. This process will take some time to be finished for the whole area of interest.

In [21]:
dates = [d.strftime("%Y-%m-%d") for d in pd.date_range(start = start_date,
                                                       end = end_date,
                                                       freq = '1d')]

with multiprocessing.Manager() as manager: 
    
    result_queue = manager.Queue()
    error_queue = manager.Queue()
    
    with multiprocessing.Pool(NUMBER_OF_PROCESSES) as pool:
        parameters = [(d,
                       mode,
                       visum.to_dict('records'),cycles,
                       hbefa,
                       result_queue,
                       error_queue,
                       ) for d in dates]
        
        res = pool.starmap(process_daily_emissions, parameters)
    
    # concatenate final process results.
    result = result_queue.get() #get first result from queue
    while not result_queue.empty():
        print('Concatenate final process results')
        new_result = result_queue.get()
        for road_index, emissions in result.items():
            for component, value in emissions.items():
                add_emissions = new_result[road_index][component]
                result[road_index][component] += add_emissions
                
    # retrieve process errors
    errors = list()
    while not error_queue.empty(): 
        errors.append(error_queue.get())

Finished calculating 2019-01-01


## Print Errors

In [6]:
for e in errors:
    print (e)

## Concatenate Results
All results are saved in result dict. This can be appended to the traffic model. 

In [9]:
# concatenate results and multiply with road length
result_df = pd.DataFrame(result).transpose()
result_df.columns = result_df.columns.map('_'.join)
visum_result = pd.concat([visum, result_df], axis = 1)

## Store results

In [21]:
if store_results: 
    
    path = data_paths.INVENTORY_FOLDER_PATH
    visum_result.to_file(path + store_filename, driver='GPKG')

# Calculate and Save Timeprofiles

In [101]:
if calculate_timeprofile: 
    
    components = ['CO2(rep)', 'NOx', 'CO'] # only calculate timeprofiles for these emission components
    
    # timeframe of interest
    dates = [d.strftime("%Y-%m-%d") for d in pd.date_range(start = start_date,
                                                        end = end_date,
                                                        freq = '1d')]

    #placeholder for raw temporal profile
    raw_profile = pd.DataFrame()
    for day in dates:
        em_dict = process_hourly_emissions(day,
                                        visum[visum['road_type'] != 'Access-residential'].to_dict('records'), # reduce complexity
                                        cycles,
                                        hbefa)

        em_sum = pd.DataFrame(em_dict).sum(axis = 1).reset_index()
        em_sum.columns = ['vehcat', 'component', 'hour', 'emission']
        em_fin = em_sum.groupby(['component', 'hour']).sum(numeric_only=True).reset_index()
        em_fin['date'] = day
        raw_profile = pd.concat([raw_profile, em_fin], axis = 0)
        print('finished day', day)

# add timestamp and year to raw profile     
raw_profile['timestamp'] = pd.to_datetime(raw_profile['date'] + ' ' + raw_profile['hour'].astype(str) + ':00:00')
raw_profile['year'] = raw_profile['timestamp'].dt.year

# convert raw profile into scaling factors by dividing by mean emission
temporal_profile = pd.DataFrame()
for idx, grp in raw_profile.groupby(['component', 'year']):
    grp['scaling_factor'] = grp['emission'] / grp['emission'].mean()
    temporal_profile = pd.concat([temporal_profile, grp[['year', 'component', 'timestamp', 'scaling_factor']]], axis = 0)
    
# store temporal profiles
if store_timeprofiles: 
    store_path = data_paths.INVENTORY_FOLDER_PATH +'/temporal_profiles/'

    # store individual file for each year
    for (year, component), data in temporal_profile.groupby(['year', 'component']):
        temporal_profile.to_csv(store_path + f'temporal_profile_{component}_{year}.csv', index = False)

finished day 2019-01-01
finished day 2019-01-02
finished day 2019-01-03
finished day 2019-01-04
finished day 2019-01-05
finished day 2019-01-06
finished day 2019-01-07
finished day 2019-01-08
finished day 2019-01-09
finished day 2019-01-10
finished day 2019-01-11
finished day 2019-01-12
finished day 2019-01-13
finished day 2019-01-14
finished day 2019-01-15
finished day 2019-01-16
finished day 2019-01-17
finished day 2019-01-18
finished day 2019-01-19
finished day 2019-01-20
finished day 2019-01-21
finished day 2019-01-22
finished day 2019-01-23
finished day 2019-01-24
finished day 2019-01-25
finished day 2019-01-26
finished day 2019-01-27
finished day 2019-01-28
finished day 2019-01-29
finished day 2019-01-30
finished day 2019-01-31
finished day 2019-02-01
finished day 2019-02-02
finished day 2019-02-03
finished day 2019-02-04
finished day 2019-02-05
finished day 2019-02-06
finished day 2019-02-07
finished day 2019-02-08
finished day 2019-02-09
finished day 2019-02-10
finished day 201